In [19]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [20]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

# TIME_FRAMES = 50
BUFFER_SIZE = 100
# AMOUNT_OF_FRAMES = TIME_FRAMES - BUFFER_SIZE + 1
# AMOUNT_OF_VIDEOS = int(input("amount of videos: "))

WIDTH = 1280
HEIGHT = 720

frame_sq = []
frame_sq_gray = []
frame_sq_edges = []

# dataset_no = input("starting number: ")
video_count = 0
                    



# rgb = np.ndarray((TIME_FRAMES - (TIME_FRAMES % BUFFER_SIZE), 3))



In [21]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
#     dataset.remove(dataset[0])
#     for datum in data:
#         dataset = np.append(dataset, datum, axis = 1)
#     result  = np.append(data[0],np.append(dataset[1],np.append(dataset[2],dataset[3],axis=1),axis=1),axis=1)
    result = np.append(h,np.append(s,np.append(v,opticFlow,axis=1),axis=1),axis=1)
    return result
def createTSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "dataset" + str(dataset_no) + ".tsv"
    datasetName = filename + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "label" + str(dataset_no) + ".tsv"
    datasetName = filename
    file = open(datasetName, "w")
    file.write("havg\thv\tsavg\tsv\tvavg\tvv\tmagavg\tmagv\tangavg\tangv\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return


<h1>Normalization</h1>

In [ ]:
from sklearn.preprocessing import StandardScaler
from numpy import genfromtxt
dataset = (genfromtxt("./unnormFrankensteinFullForm.csv", delimiter=','))[1:,:]
scaler = StandardScaler(copy=True)
print(scaler.fit(dataset))
def normalize(dataset):
    datasetTmp = scaler.transform(dataset)
    return datasetTmp

In [23]:
def createCSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     dataset_name = "dataset" + str(dataset_no) + ".csv"
    dataset_name = filename + ".csv"
    csv = open(dataset_name, "w")
    columnTitleRow = "havg,hv,savg,sv,vavg,vv,magavg,magv,angavg,angv\n"
    csv.write(columnTitleRow)
    for i in range(dataset.shape[0]):
        csv.write(",".join(dataset[i,:].tolist()))
        csv.write("\n")
    csv.close()

<h1>STORE VIDEOS</h1>

In [ ]:
import os
import cv2
import sys

directory_name = input("directory name: ")
os.makedirs(directory_name)

BUFFER_SIZE = 100
# AMOUNT_OF_VIDEOS = 70
RECORD_AMOUNT = 20
video_count = 91

for i in range(RECORD_AMOUNT + 1):
#     print("recording video " + str(video_count) + "...")
    sys.stdout.write("\r" + "recording video " + str(video_count) + "...")
    sys.stdout.flush()
    cap = cv2.VideoCapture(0)
    WIDTH = int(cap.get(3))
    HEIGHT = int(cap.get(4))
    video_name = directory_name + "/" + str(video_count) + ".avi"
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(directory_name + "/" + str(video_count) + '.avi',fourcc, 20.0, (640,480))
    frame_counter = 0
    while(True):
        ret, frame = cap.read()
        if (ret == True):
            out.write(frame)
#             cv2.imshow('frame',frame)
            frame_counter += 1
        if (frame_counter == BUFFER_SIZE): break
    video_count += 1
#     tmp = input("press enter to continue")
    cap.release()
    cv2.destroyAllWindows()
    tmp = input("press Enter")
video_count = 0

    
    

<h1>ANALYZE VIDEOS + PREPROCESSING</h1>

In [24]:
import os
import sys

directory = input('enter folder name: ')
vids = os.listdir(directory)
x = 0
BUFFER_SIZE = 100
tmp = np.arange(x,len(vids) - 1 + x)
videos = [name + '.avi' for name in (tmp.astype(str)).tolist()]
videos = [directory + "/" + name for name in videos]
video_count = 0
frame_count = 0
AMOUNT_OF_VIDEOS = len(videos) - 1

dataset = np.ndarray((AMOUNT_OF_VIDEOS, BUFFER_SIZE - 1, HEIGHT//2, WIDTH//2, 5))
opticflow_buffer = []

print("then")
while(video_count < len(videos)-1):
# while(True):
#     print("video " + str(video_count) + "...")
    sys.stdout.write("\r" + "video " + str(videos[video_count]) + "...")
    sys.stdout.flush()
    cap = cv2.VideoCapture(videos[video_count])
    for i in range (BUFFER_SIZE):
        ret, frame = cap.read()
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        opticflow_buffer.append(gray)
        if(len(opticflow_buffer) == 2):
            hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
            flow = cv2.calcOpticalFlowFarneback(opticflow_buffer[0],opticflow_buffer[1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
            del opticflow_buffer[0]
            print(hsv.shape)
            print(flow.shape)
            data_frame = np.stack((hsv,flow),axis=-1)
            dataset[video_count, frame_count, :, :, :]
            frame_count += 1
    cap.release()
    cv2.destroyAllWindows()
    opticflow_buffer = []
    video_count += 1
# datasetUnnormalized = createDataset([h,s,v,opticFlow])
cv2.destroyAllWindows()

MemoryError: 

In [36]:
datasetUnnormalized = createDataset([h,s,v,opticFlow])
print(datasetUnnormalized.shape)
print(videos)

(18, 10)
['addition/71.avi', 'addition/72.avi', 'addition/73.avi', 'addition/74.avi', 'addition/75.avi', 'addition/76.avi', 'addition/77.avi', 'addition/78.avi', 'addition/79.avi', 'addition/80.avi', 'addition/81.avi', 'addition/82.avi', 'addition/83.avi', 'addition/84.avi', 'addition/85.avi', 'addition/86.avi', 'addition/87.avi', 'addition/88.avi', 'addition/89.avi']


In [37]:
# (normalized,nans) = normalize(datasetUnnormalized)
(normalized) = normalize(datasetUnnormalized)

In [38]:
# print(normalized)
# print(nans)
# print(normalize(datasetUnnormalized))
createCSV(datasetUnnormalized,"unnormre1")
createCSV(normalized,"normre1")
createTSV(normalized,"normre1")

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(datasetUnnormalized)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [30]:
# print(normalize(datasetUnnormalized))
# print(datasetUnnormalized)
data = datasetUnnormalized[5,:]
data[3] = 1.95701144e+05
print(data)
data = data.reshape(1,-1)
print(scaler.transform(data))

[2.34587818e-01 1.17650179e+00 2.76209289e-01 1.95701144e+05
 4.76514989e-01 6.75283781e+00 1.74247715e-05 3.90512085e-04
 2.19686303e-03 1.01534326e-01]
[[-4.01805330e-01 -1.00193358e+00 -4.81918523e-01  5.22244023e+04
   6.36985515e-01 -1.02634940e+00  1.57668432e+00 -1.12283394e+00
  -1.87308162e-01  1.19051603e-01]]


In [ ]:
a = scaler.transform(datasetUnnormalized)
a

<h1>DATASET APPEND</h1>

In [1]:
import numpy as np
from numpy import genfromtxt
def appendDatasets():
    dataset = []
    dataset_counter = 0
    pathName = input("Path name: ")
    while(True):
        fileName = input("File name: ")
        if(fileName == ""):
            break
        next_dataset = (genfromtxt(pathName + fileName, delimiter=','))[1:,:]
        if(dataset_counter == 0):
            dataset = next_dataset
            dataset_counter += 1
        else:
            dataset = np.append(dataset, next_dataset, axis = 0)
    return dataset
def appendLabels():
    labels = []
    pathName = input("Path name: ")
    while(True):
        fileName = input("File name: ")
        if(fileName == ""):
            break
        with open(pathName + fileName) as f:
            lines = f.readlines()
        for line in lines:
            labels.append(line[:-1])
    return labels
def saveLabels(label):
#     pathName = input("Path name: ")
    fileName = input("File name: ")
    import pickle
    pickle.dump(label,open(fileName + ".p", "wb"))
def makeTSVLabels(labels):
    fileName = input("File name: ")
    file = open(fileName + ".tsv", "w")
    for i in range(len(labels)):
        file.write(labels[i] + "\n")
    return


In [6]:
dts = appendDatasets()
# createCSV(dts,"normres")

In [12]:
createTSV(dts,"unnormres")

In [13]:
createCSV(dts,"unnormres")

In [16]:
normdts = normalize(dts)

minn:  [1.68698948e-01 5.71359754e-01 7.26006841e-02 1.91583500e-01
 2.30490743e-01 1.69076206e+00 1.60691440e-06 3.60041682e-04
 7.00276490e-04 5.24942415e-02]
rangee:  [1.88006912e-01 2.37081875e+00 5.25229358e-01 1.84091510e+01
 3.98871483e-01 1.38006918e+01 9.30031806e-05 1.20997849e-03
 4.86319761e-03 7.18616676e-02]


In [17]:
createCSV(normdts,"normFrankensteinFullForm")

In [21]:
createTSV(dts,"FrankensteinFullForm")

In [11]:
labels = appendLabels()
print(labels)
saveLabels(labels)

['static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'static', 'rot', 'rot', 'rot', 'rot', 'rot', 'rot', 'rot', 'rot', 'static', 'static', 'rot', 'rot', 'frot', 'frot', 'frot', 'light', 'light', 'light', 'light', 'light', 'light', 'light', 'unknown', 'light', 'unknown', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'frot', 'frot', 'frot', 'frot', 'frot', 'static', 'frot', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'move', 'light', 'light', 'light', 'light', 'unknown', 'light', 'light', 'light', 'light', 'light', 'light', 'light', 'light', 'light', 'unknown', 'light', 'light', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'dynamiclight', 'd

In [26]:
import psutil
psutil.virtual_memory()

svmem(total=33641873408, available=27199905792, percent=19.1, used=8269729792, free=20736163840, active=7113564160, inactive=5019258880, buffers=653795328, cached=3982184448, shared=290054144, slab=381558784)